# Time Serise Algorithm for Sentinel-2 Cloud Mask

## Overview
This product is a time series cloud and cloud shadow detection algorithm for Sentinel-2 surface reflectance data.It models time series of surface reflectance derived indices and calculates time series abnormality coefficients for pixels in the time series. It does not rely on predefined training data to generate complex models with many rule sets, which often work well for data similar to the training data while returning poor results for data contrasting to the training data. Instead, it identifies cloud and cloud shadows by detecting local abnormalities in temporal and spatial contexts from abnormality coefficients.


## Required Modules 

### DEA modules


The program requires functions in DEA Datacube API. To load DEA module, open a terminal and run following commands: 

`module use /g/data/v10/public/modules/modulefiles`

`module load dea`

### Other Python modules 


## Input Data



## Workflow and Functions of the Program  

### Data loading




### Cloud and Cloud shadow detection



## Output Data

The program produces a cloud mask data with the same dimension as the input time series data







**Function ID: F** 

**Function Name:**

**Description:** 


This is the template for function description

**Input:**

**Return:**

**called by:**


**Function ID: F1** 

**Function Name:**

**Description:** 


This function load selected Sentinel-2 Surface reflectance data. The spatial coverage is specfied by a rectangle bounding box. The temporal coverage is specified by the start date and the end date of the time series data

**Input:**

The lat./Lon. of the top-left and bottom-right corners of a b 

**Return:**

**called by:**


**Function ID: F2** 

**Function Name:**

**Description:** 


This function load pre-saved Sentinel-2 Surface reflectance data, specified by the name of the data file

**Input:**

**Return:**

**called by:**




**Function ID: F3** 

**Function Name:**

**Description:** 


This function detect cloud and cloud shadows using time series of Sentinel-2 surface reflectance data




**Input:**

**Return:**

**called by:**


In [ ]:
## Main program

## Import modules 

## Specify input parameters

## Load surface reflectance data

## Call F1 or F2


## Divide the whole data into small chunks, use xarray new(v0.16) map_block function
## 


## Call time series cloud / shadow detection for each chunk  



## Apply spatial cloud filter


## Output cloud/shadow mask results




## End Main program


In [ ]:
## Function Id: F1
## Description:
## Input:
## Return:
## Called by:


In [ ]:
## Function Id: F1
## Description:
## Input:
## Return:
## Called by:


In [ ]:
## Function Id: F1
## Description:
## Input:
## Return:
## Called by:
